### Домашнее задание
Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
Повторить п.2, но используя уже не медиану, а max
(опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных
Ссылки
http://www.machinelearning.ru/wiki/images/d/d5/Voron17survey-artm.pdf
https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation
Библиотеки, которые нужно установить:
gensim
razdel
pymorphy2
nltk

In [3]:
!pip install gensim
!pip install razdel
!pip install pymorphy2
!pip install nltk

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13709 sha256=80c7e26233f8f4f1aa0a54b7dd6636505232f7561e47744349710884b9b86be1
  Stored in directory: c:\users\матвей\appdata\local\pip\cache\wheels\56\ea\58\ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [5]:
!pip install lightgbm

In [6]:
import pandas as pd
import numpy as np
import scipy.sparse

from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import json

from datetime import datetime

import lightgbm as lgb
import gc

In [56]:
news = pd.read_csv('./articles.csv', quotechar='\"', 
                 escapechar='\\', 
                 error_bad_lines=False, nrows=800000)
news = news[~news['title'].isnull()][['edition', 'title']]
news['edition'] = range(len(news))
news.index = range(len(news))
news.head(3)

KeyError: "['edition'] not in index"

In [46]:
news['edition'].value_counts()

Series([], Name: doc_id, dtype: int64)

In [47]:
news = pd.concat([news[news['edition']==u'Спорт'].iloc[:3000], 
                  news[news['edition']==u'Политика'].iloc[:3000],
                  news[news['edition']==u'Культура'].iloc[:3000], 
                  news[news['edition']==u'Армия'].iloc[:3000],
                  news[news['edition']==u'Стиль'].iloc[:3000], 
                  news[news['edition']==u'Общество'].iloc[:3000],
                  news[news['edition']==u'Бизнес'].iloc[:3000], 
                  news[news['edition']==u'Новости'].iloc[:3000],
                  news[news['edition']==u'Наука'].iloc[:3000]])

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [48]:
news[['doc_id', 'title']].to_csv("articles.csv", index=None)

In [49]:
users = ["u10{}".format(i) for i in range(1000, 9000)]
users = pd.DataFrame({'uid': users, 'churn': [0 for i in range(7000)]+[1 for i in range(1000)]})
users.sample(frac=1).to_csv("users_churn.csv", index=None)
users.head(3)

,uid,churn
0,u101000,0
1,u101001,0
2,u101002,0


In [50]:
news['doc_id'].value_counts()

Series([], Name: doc_id, dtype: int64)

In [52]:
sport = news[news['doc_id']==u'Спорт']['doc_id'].values
politics = news[news['doc_id']==u'Политика']['doc_id'].values
culture = news[news['doc_id']==u'Культура']['doc_id'].values
business = news[news['doc_id']==u'Бизнес']['doc_id'].values
army = news[news['doc_id']==u'Армия']['doc_id'].values
society = news[news['doc_id']==u'Общество']['doc_id'].values
news_ = news[news['doc_id']==u'Новости']['doc_id'].values
science = news[news['doc_id']==u'Наука']['doc_id'].values

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [53]:
news

,doc_id,title


In [54]:
total = []
for i in range(2000):
    q = np.random.choice(sport, 5).tolist()+np.random.choice(science, 1).tolist()
    total.append(q)
for i in range(2000):
    q = np.random.choice(politics, 5).tolist()+np.random.choice(news_, 1).tolist()
    total.append(q)
for i in range(1000):
    q = np.random.choice(culture, 5).tolist()+np.random.choice(society, 1).tolist()
    total.append(q)
for i in range(1000):
    q = np.random.choice(politics, 5).tolist()+np.random.choice(society, 1).tolist()
    total.append(q)
for i in range(1000):
    q = np.random.choice(politics, 5).tolist()+np.random.choice(army, 1).tolist()
    total.append(q)
for i in range(1000):
    q = np.random.choice(society, 5).tolist()+np.random.choice(politics, 1).tolist()
    total.append(q)

ValueError: 'a' cannot be empty unless no samples are taken

In [26]:
pd.DataFrame({'uid': ["u10{}".format(i) for i in range(1000, 9000)], 
              'articles': total}).sample(frac=1).to_csv("users_articles.csv", index=None)

NameError: name 'total' is not defined

In [27]:
news[news['doc_id']==492909]

,doc_id,title


In [14]:
pd.DataFrame({'uid': ["u10{}".format(i) for i in range(1000, 9000)], 
              'articles': total}).sample(frac=1)

,uid,articles
4263,u105263,"[295284, 294492, 290056, 294886, 293817, 1093]"
1873,u102873,"[5780, 5117, 7010, 7823, 6333, 513172]"
6613,u107613,"[323409, 324675, 323532, 324015, 322666, 470715]"
7198,u108198,"[2579, 1864, 2312, 703, 2540, 322150]"
5301,u106301,"[323951, 322288, 323440, 323377, 323651, 2416]"
...,...,...
7934,u108934,"[3322, 2279, 1202, 1819, 666, 324212]"
4646,u105646,"[295249, 293955, 293981, 295057, 294737, 620]"
2274,u103274,"[324294, 323729, 322282, 322723, 322368, 513324]"
3977,u104977,"[324098, 322637, 322263, 323564, 323256, 519863]"
